In [ ]:
# Este entorno Python 3 viene con muchas bibliotecas de análisis útiles instaladas
# Está definido por la imagen de Docker de kaggle/python: https://github.com/kaggle/docker-python
# Por ejemplo, aquí hay varios paquetes útiles para cargar

import numpy as np
import pandas as pd

# Los archivos de datos de entrada están disponibles en el directorio de solo lectura "../input/"
# Por ejemplo, ejecutar esto (haciendo clic en ejecutar o presionando Shift+Enter) enumerará todos los archivos en el directorio de entrada

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/bali-popular-destination-for-tourist-2022/Bali Popular Destination for Tourist 2022 - Sheet1.csv


In [ ]:
import os
import sketch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from dataprep.eda import plot, plot_missing, create_report

In [ ]:
data = pd.read_csv('/kaggle/input/bali-popular-destination-for-tourist-2022/Bali Popular Destination for Tourist 2022 - Sheet1.csv')

In [ ]:
data.head()

,Place,Location,Coordinate,Google Maps Rating,Google Reviews (Count),Source,Description,Tourism/Visitor Fee (approx in USD)
0,Tanah Lot,"Beraban, Kec. Kediri, Kabupaten Tabanan, Bali ...","8.6212° S, 115.0868° E",4.6,75899,https://www.google.com/maps/place/Tanah+Lot/@-...,is a rock formation off the Indonesian island ...,"Yes, entrance tickets cost 20,000 Rupiah for I..."
1,Mount Batur,"Bangli Regency, Bali, Indonesia",8°14′20″S 115°22′39″E,4.5,2580,https://www.google.com/maps/place/Mt+Batur/@-8...,Mount Batur (Gunung Batur) is an active volcan...,"Yes, The Mount Batur trekking price is around ..."
2,Uluwatu Temple,"Uluwatu (South Kuta, Badung)","8.8291° S, 115.0849° E",4.6,28800,https://www.google.com/maps/place/Uluwatu+Temp...,The temple is regarded as one of the sad kahya...,"Yes, The Uluwatu Temple Bali entrance fee is I..."
3,Ubud Monkey Forest,"Jl. Monkey Forest, Ubud, Kecamatan Ubud, Kabup...","8.5194° S, 115.2606° E",4.5,36099,https://www.google.com/maps/place/Sacred+Monke...,"Mandala Suci Wenara Wana, or well known as Ubu...","Yes, Visit to the Ubud Monkey Forest are subje..."
4,Goa Gajah,"Gianyar Regency, Bali","8.5069° S, 115.2625° E",4.2,6683,https://www.google.com/maps/place/Goa+Gajah/@-...,"Goa Gajah, or Elephant Cave, is located on the...","Yes, Adult 50.000 IDR ($3.4 USD), Child 25.000..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Place                                34 non-null     object 
 1   Location                             34 non-null     object 
 2   Coordinate                           34 non-null     object 
 3   Google Maps Rating                   34 non-null     float64
 4   Google Reviews (Count)               34 non-null     int64  
 5   Source                               34 non-null     object 
 6   Description                          34 non-null     object 
 7   Tourism/Visitor Fee (approx in USD)  34 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 2.2+ KB


In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Google Maps Rating,34.0,4.505882,0.143424,4.2,4.4,4.5,4.6,4.8
Google Reviews (Count),34.0,15270.294118,16131.954807,172.0,4902.0,9345.5,16352.5,75899.0


In [ ]:
data.describe(include='O').T

,count,unique,top,freq
Place,34,34,Tanah Lot,1
Location,34,33,"Kuta, Badung Regency, Bali",2
Coordinate,34,34,"8.6212° S, 115.0868° E",1
Source,34,34,https://www.google.com/maps/place/Tanah+Lot/@-...,1
Description,34,34,is a rock formation off the Indonesian island ...,1
Tourism/Visitor Fee (approx in USD),34,29,"Yes, depending on location and places visited.",4


In [ ]:
from dataprep.clean import clean_lat_long

clenedDf = clean_lat_long(data, "Coordinate", split=True)


/opt/conda/lib/python3.10/site-packages/dask/dataframe/core.py:7087: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



  0%|          | 0/9 [00:00<?, ?it/s]

Latitude and Longitude Cleaning Report:
	34 values cleaned (100.0%)
Result contains 34 (100.0%) values in the correct format and 0 null values (0.0%)


In [ ]:

def find_and_filter_numbers(df, column_name):
    """
    Mencari angka dalam kolom teks DataFrame Pandas dan mengambil angka
    dengan ukuran tidak lebih dari 2.

    Parameters:
    df (pd.DataFrame): DataFrame Pandas yang berisi data teks.
    column_name (str): Nama kolom dalam DataFrame yang ingin dianalisis.

    Returns:
    pd.Series: Seri Pandas berisi angka dengan ukuran tidak lebih dari 2.
    """
 # Buscar números en una columna usando expresiones regulares
    pattern = r'\$ \d+'
    df[column_name] = df[column_name].astype(str)
    df['numbers'] = df[column_name].apply(lambda x: re.findall(pattern, x))

    return df['numbers']

In [ ]:
fees_list = find_and_filter_numbers(clenedDf, 'Tourism/Visitor Fee (approx in USD)')

from collections import Counter

filtered_fees = dict(Counter([fe for fee in fees_list for fe in fee]))

In [ ]:
data.head()

,Place,Location,Coordinate,Google Maps Rating,Google Reviews (Count),Source,Description,Tourism/Visitor Fee (approx in USD)
0,Tanah Lot,"Beraban, Kec. Kediri, Kabupaten Tabanan, Bali ...","8.6212° S, 115.0868° E",4.6,75899,https://www.google.com/maps/place/Tanah+Lot/@-...,is a rock formation off the Indonesian island ...,"Yes, entrance tickets cost 20,000 Rupiah for I..."
1,Mount Batur,"Bangli Regency, Bali, Indonesia",8°14′20″S 115°22′39″E,4.5,2580,https://www.google.com/maps/place/Mt+Batur/@-8...,Mount Batur (Gunung Batur) is an active volcan...,"Yes, The Mount Batur trekking price is around ..."
2,Uluwatu Temple,"Uluwatu (South Kuta, Badung)","8.8291° S, 115.0849° E",4.6,28800,https://www.google.com/maps/place/Uluwatu+Temp...,The temple is regarded as one of the sad kahya...,"Yes, The Uluwatu Temple Bali entrance fee is I..."
3,Ubud Monkey Forest,"Jl. Monkey Forest, Ubud, Kecamatan Ubud, Kabup...","8.5194° S, 115.2606° E",4.5,36099,https://www.google.com/maps/place/Sacred+Monke...,"Mandala Suci Wenara Wana, or well known as Ubu...","Yes, Visit to the Ubud Monkey Forest are subje..."
4,Goa Gajah,"Gianyar Regency, Bali","8.5069° S, 115.2625° E",4.2,6683,https://www.google.com/maps/place/Goa+Gajah/@-...,"Goa Gajah, or Elephant Cave, is located on the...","Yes, Adult 50.000 IDR ($3.4 USD), Child 25.000..."
